<a href="https://colab.research.google.com/github/coll-j/B21-CAP0001/blob/ml/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re

from tensorflow.keras.layers import Dense, RNN, LSTM

In [ ]:
!wget https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/re_dataset.csv

--2021-05-22 11:57:28--  https://raw.githubusercontent.com/okkyibrohim/id-multi-label-hate-speech-and-abusive-language-detection/master/re_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1858473 (1.8M) [text/plain]
Saving to: ‘re_dataset.csv’

re_dataset.csv      100%[===================>]   1.77M  --.-KB/s    in 0.07s   

2021-05-22 11:57:28 (25.4 MB/s) - ‘re_dataset.csv’ saved [1858473/1858473]



In [ ]:
df = pd.read_csv('re_dataset.csv', encoding="ISO-8859-1")
df.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [ ]:
df['Tweet'][5]

"USER Ya bani taplak dkk \\xf0\\x9f\\x98\\x84\\xf0\\x9f\\x98\\x84\\xf0\\x9f\\x98\\x84'"

In [ ]:
df['Tweet'][8]

'Nah admin belanja satu lagi port terbaik nak makan Ais Kepal Milo, Ais Kepal Horlicks atau Cendol Toping kaw kaw. ð??\xad; ; Doket mano tu ? Gerai Rojak Mertuaku - Taipan 2 (depan TWINS BABY &amp; ROMANTIKA / Bank Islam Senawang) ð???'

In [ ]:
len(df)

13169

# Pre-Processing

## Cleaning text

In [ ]:
tweets = df['Tweet'].values.astype(str)
tweets

array(["- disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego ! ! !'",
       "RT USER: USER siapa yang telat ngasih tau elu?edan sarap gue bergaul dengan cigax jifla calis sama siapa noh licew juga'",
       '41. Kadang aku berfikir, kenapa aku tetap percaya pada Tuhan padahal aku selalu jatuh berkali-kali. Kadang aku merasa Tuhan itu ninggalkan aku sendirian. Ketika orangtuaku berencana berpisah, ketika kakakku lebih memilih jadi Kristen. Ketika aku anak ter',
       ...,
       "USER Hati hati bisu :( .g\\n\\nlagi bosan huft \\xf0\\x9f\\x98\\xaa'",
       "USER USER USER USER Bom yang real mudah terdeteksi bom yang terkubur suatu saat lebih dahsyat ledakannya itulah di sebut Revolusi Jiwa'",
       'USER Mana situ ngasih(": itu cuma foto ya kutil onta\''],
      dtype='<U561')

In [ ]:
query = r'USER[:]? |\\[\\A-Za-z0-9]*' # Hilangkan 'USER' dan encoding emoji ex: '\\xf0\\xxx\\xxx...'
tweets = list(map(lambda s: re.sub(query, '', s), tweets))
query = r'[^a-zA-Z\d\s]' # ambil alphanumeric saja
tweets = list(map(lambda s: re.sub(query, '', s), tweets))
tweets = np.char.strip(tweets) # hilangkan trailing spaces
tweets

array(['disaat semua cowok berusaha melacak perhatian gue loe lantas remehkan perhatian yg gue kasih khusus ke elo basic elo cowok bego',
       'RT siapa yang telat ngasih tau eluedan sarap gue bergaul dengan cigax jifla calis sama siapa noh licew juga',
       '41 Kadang aku berfikir kenapa aku tetap percaya pada Tuhan padahal aku selalu jatuh berkalikali Kadang aku merasa Tuhan itu ninggalkan aku sendirian Ketika orangtuaku berencana berpisah ketika kakakku lebih memilih jadi Kristen Ketika aku anak ter',
       ..., 'Hati hati bisu  g bosan huft',
       'Bom yang real mudah terdeteksi bom yang terkubur suatu saat lebih dahsyat ledakannya itulah di sebut Revolusi Jiwa',
       'Mana situ ngasih itu cuma foto ya kutil onta'], dtype='<U275')

In [ ]:
len(tweets)

13169

## Tokenize